<h1>DS Capstone Project: Location Segmentation with K Means</h1>

<h2>Question 1: pre-processing postal data</h2>

In [12]:
# import basics, and load postal data into pd dataframe called postal_data
import pandas as pd
import numpy as np
postal_data = pd.read_csv('./data/postal_data.csv')
postal_data.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [13]:
# as not assigned Borough lines had been deleted in csv, now only need to check those with not assigned neighbourhodd
# and assign Borough value to neighbourhood
check_NA = postal_data[postal_data['Neighbourhood']=="Not assigned"]
check_NA

,Postcode,Borough,Neighbourhood
6,M9A,Queen's Park,Not assigned


In [14]:
postal_data.loc[postal_data['Neighbourhood']=="Not assigned"]=postal_data['Borough']
# and check again
check_NA = postal_data[postal_data['Neighbourhood']=="Not assigned"]
check_NA

,Postcode,Borough,Neighbourhood


In [39]:
# group by Post code:
postal_data = pd.DataFrame(postal_data.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: ', '.join(x))).reset_index()
postal_data

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9P,Etobicoke,Westmount
99,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
100,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."
101,M9W,Etobicoke,Northwest


In [40]:
postal_data.shape

(103, 3)

<h2>Question 2: Get Latitude and Longitude</h2>

In [41]:
!conda install -c conda-forge geocoder --yes 
!conda install -c conda-forge geopy --yes 

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\320069333\AppData\Local\Continuum\anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          92 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.21.0-py_0




geographiclib-1.50   | 34 KB     |            |   0% 
geographiclib-1.50   | 34 KB     | ####7      |  47% 
geographiclib-1.50   | 34 KB     | ########## | 100% 

geopy-1.21.0         | 58 KB     |            |   0% 

In [52]:
import pandas as pd

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="nnn")

from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
postal_data['Location'] = postal_data['Postcode'].apply(geolocator.geocode, timeout=15)

In [57]:
postal_data['Location'] = postal_data['Location'].apply(lambda x: x[9][0])

Location(M1B, K Letišti, Ruzyně, Praha, okres Hlavní město Praha, Hlavní město Praha, Praha, 16100, Česká republika, (50.1005945, 14.2872824, 0.0))

In [63]:
# Geocoder not accessable; GeoPy provides the wrong location information, so I am gonna load the geo data from csv

geo_data = pd.read_csv('./data/Geospatial_Coordinates.csv')
geo_data

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [71]:
final_data = pd.merge(postal_data[['Postcode','Borough','Neighbourhood']],
                      geo_data,
                      left_on='Postcode',
                      right_on='Postal Code',
                      how='left'
                     )
final_data.drop(['Postal Code'], axis=1, inplace = True)
final_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


<h2>Question 3: data exploration and segmentation</h2>

In [75]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\320069333\AppData\Local\Continuum\anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.0.1               |             py_0         575 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    certifi-2019.9.11          |           py37_0         147 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         820 KB

The following NEW packages will be INSTALLED:

  al

In [76]:
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [72]:
# create map around center of locations
latitude = final_data['Latitude'].mean()
longitude = final_data['Longitude'].mean()
print(latitude,longitude)

43.704968049019605 -79.39582850490194


In [82]:
final_data.dropna(inplace=True)

In [83]:
mymap = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(final_data['Latitude'], final_data['Longitude'], final_data['Borough'], final_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(mymap)  
mymap

In [86]:
# choose Scarborough to zoom in:
s_data = final_data[final_data['Borough'] == 'Scarborough'].reset_index(drop=True)
s_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [88]:
address = 'Scarborough, Canada'
# as geocoder service timeout, use average location
latitude = s_data['Latitude'].mean()
longitude = s_data['Longitude'].mean()
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 43.76622889411766, -79.24908523529415.


In [89]:
CLIENT_ID = 'UNB1ZPJYRJ4XHM31H2E3SVOUAMGAPRXK0H1S11SWHZSU4OOO' # your Foursquare ID
CLIENT_SECRET = '2JOEJTF4UXPM1HZPJL1GBZLUPTZNUSG4RGBVABVUH52PLMR0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UNB1ZPJYRJ4XHM31H2E3SVOUAMGAPRXK0H1S11SWHZSU4OOO
CLIENT_SECRET:2JOEJTF4UXPM1HZPJL1GBZLUPTZNUSG4RGBVABVUH52PLMR0


In [99]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=10):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        res = requests.get(url).json()
        if res["response"]['groups']:
            results = res["response"]['groups'][0]['items']

            # return only relevant information for each nearby venue
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [100]:
nn = getNearbyVenues(names=s_data['Neighbourhood'], latitudes=s_data['Latitude'], longitudes=s_data['Longitude'], radius=500, LIMIT=10)
nn.head()

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa


In [101]:
# one hot encoding
onehot = pd.get_dummies(nn[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] = nn['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Brewery,Bus Line,...,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Spa,Thai Restaurant,Vietnamese Restaurant
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [102]:
grouped = onehot.groupby('Neighborhood').mean().reset_index()
grouped

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Brewery,Bus Line,...,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Spa,Thai Restaurant,Vietnamese Restaurant
0,Agincourt,0.0,0.000,0.000,0.000,0.000,0.0,0.200,0.000000,0.000,...,0.000,0.0,0.000,0.000,0.000,0.20,0.000,0.000,0.000,0.000000
1,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.000,0.000,0.000,0.000,0.0,0.000,0.000000,0.000,...,0.000,0.5,0.000,0.000,0.000,0.00,0.000,0.000,0.000,0.000000
2,"Birch Cliff, Cliffside West",0.0,0.000,0.000,0.000,0.000,0.0,0.000,0.000000,0.000,...,0.000,0.0,0.000,0.000,0.000,0.25,0.000,0.000,0.000,0.000000
3,Cedarbrae,0.0,0.125,0.000,0.125,0.125,0.0,0.000,0.000000,0.000,...,0.000,0.0,0.000,0.000,0.000,0.00,0.000,0.000,0.125,0.000000
4,"Clairlea, Golden Mile, Oakridge",0.0,0.000,0.000,0.250,0.000,0.0,0.000,0.000000,0.125,...,0.000,0.0,0.000,0.000,0.000,0.00,0.125,0.000,0.000,0.000000
5,"Clarks Corners, Sullivan, Tam O'Shanter",0.0,0.000,0.000,0.000,0.100,0.0,0.000,0.000000,0.000,...,0.100,0.0,0.000,0.000,0.000,0.00,0.000,0.000,0.100,0.000000
6,"Cliffcrest, Cliffside, Scarborough Village West",0.5,0.000,0.000,0.000,0.000,0.0,0.000,0.000000,0.000,...,0.000,0.0,0.000,0.000,0.000,0.00,0.000,0.000,0.000,0.000000
7,"Dorset Park, Scarborough Town Centre, Wexford ...",0.0,0.000,0.000,0.000,0.000,0.0,0.000,0.166667,0.000,...,0.000,0.0,0.000,0.000,0.000,0.00,0.000,0.000,0.000,0.166667
8,"East Birchmount Park, Ionview, Kennedy Park",0.0,0.000,0.000,0.000,0.000,0.0,0.000,0.000000,0.000,...,0.000,0.0,0.000,0.000,0.000,0.00,0.000,0.000,0.000,0.000000
9,"Guildwood, Morningside, West Hill",0.0,0.000,0.000,0.000,0.000,0.0,0.125,0.000000,0.000,...,0.125,0.0,0.125,0.000,0.000,0.00,0.000,0.125,0.000,0.000000


In [103]:
num_top_venues = 5

for hood in grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = grouped[grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0             Clothing Store   0.2
1  Latin American Restaurant   0.2
2               Skating Rink   0.2
3             Breakfast Spot   0.2
4                     Lounge   0.2


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                venue  freq
0                Park   0.5
1          Playground   0.5
2        Noodle House   0.0
3        Intersection   0.0
4  Italian Restaurant   0.0


----Birch Cliff, Cliffside West----
                   venue  freq
0        College Stadium  0.25
1  General Entertainment  0.25
2           Skating Rink  0.25
3                   Café  0.25
4    American Restaurant  0.00


----Cedarbrae----
                venue  freq
0     Thai Restaurant  0.12
1              Bakery  0.12
2                Bank  0.12
3  Athletics & Sports  0.12
4    Hakka Restaurant  0.12


----Clairlea, Golden Mile, Oakridge----
            venue  freq
0          Bakery  0.25
1  Ice Cream Shop  0.12
2          

In [104]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [112]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Clothing Store,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Vietnamese Restaurant,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
1,"Agincourt North, L'Amoreaux East, Milliken, St...",Playground,Park,Vietnamese Restaurant,Chinese Restaurant,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
2,"Birch Cliff, Cliffside West",General Entertainment,Skating Rink,College Stadium,Café,Chinese Restaurant,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
3,Cedarbrae,Hakka Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,College Stadium,General Entertainment
4,"Clairlea, Golden Mile, Oakridge",Bakery,Metro Station,Soccer Field,Ice Cream Shop,Bus Line,Bus Station,Park,Vietnamese Restaurant,Fried Chicken Joint,Fast Food Restaurant


In [113]:
# set number of clusters
kclusters = 5

grouped_clustering = grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 1, 1, 1, 1, 1, 1, 1, 1])

In [114]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

,Postcode,Borough,Neighbourhood,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",0.0,"Rouge, Malvern",Fast Food Restaurant,Vietnamese Restaurant,Thai Restaurant,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Discount Store,Department Store,Convenience Store
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",3.0,"Highland Creek, Rouge Hill, Port Union",Bar,Vietnamese Restaurant,Clothing Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",1.0,"Guildwood, Morningside, West Hill",Intersection,Rental Car Location,Medical Center,Breakfast Spot,Pizza Place,Mexican Restaurant,Electronics Store,Spa,Department Store,Discount Store
3,M1G,Scarborough,Woburn,4.0,Woburn,Coffee Shop,Korean Restaurant,Vietnamese Restaurant,Grocery Store,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
4,M1H,Scarborough,Cedarbrae,1.0,Cedarbrae,Hakka Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,College Stadium,General Entertainment


In [119]:
merged = s_data

merged = pd.merge(merged,
                      neighborhoods_venues_sorted,
                      left_on='Neighbourhood',
                      right_on='Neighborhood',
                      how='left'
                     )

merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0.0,"Rouge, Malvern",Fast Food Restaurant,Vietnamese Restaurant,Thai Restaurant,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Discount Store,Department Store,Convenience Store
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,3.0,"Highland Creek, Rouge Hill, Port Union",Bar,Vietnamese Restaurant,Clothing Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1.0,"Guildwood, Morningside, West Hill",Intersection,Rental Car Location,Medical Center,Breakfast Spot,Pizza Place,Mexican Restaurant,Electronics Store,Spa,Department Store,Discount Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,4.0,Woburn,Coffee Shop,Korean Restaurant,Vietnamese Restaurant,Grocery Store,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Cedarbrae,Hakka Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,College Stadium,General Entertainment


In [120]:
merged.drop(['Neighbourhood'], axis=1, inplace = True)
merged = merged.dropna()
merged

,Postcode,Borough,Latitude,Longitude,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,43.806686,-79.194353,0.0,"Rouge, Malvern",Fast Food Restaurant,Vietnamese Restaurant,Thai Restaurant,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Discount Store,Department Store,Convenience Store
1,M1C,Scarborough,43.784535,-79.160497,3.0,"Highland Creek, Rouge Hill, Port Union",Bar,Vietnamese Restaurant,Clothing Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
2,M1E,Scarborough,43.763573,-79.188711,1.0,"Guildwood, Morningside, West Hill",Intersection,Rental Car Location,Medical Center,Breakfast Spot,Pizza Place,Mexican Restaurant,Electronics Store,Spa,Department Store,Discount Store
3,M1G,Scarborough,43.770992,-79.216917,4.0,Woburn,Coffee Shop,Korean Restaurant,Vietnamese Restaurant,Grocery Store,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
4,M1H,Scarborough,43.773136,-79.239476,1.0,Cedarbrae,Hakka Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,College Stadium,General Entertainment
5,M1J,Scarborough,43.744734,-79.239476,2.0,Scarborough Village,Playground,Vietnamese Restaurant,Chinese Restaurant,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store
6,M1K,Scarborough,43.727929,-79.262029,1.0,"East Birchmount Park, Ionview, Kennedy Park",Discount Store,Department Store,Convenience Store,Bus Station,Coffee Shop,Vietnamese Restaurant,Clothing Store,General Entertainment,Gas Station,Fried Chicken Joint
7,M1L,Scarborough,43.711112,-79.284577,1.0,"Clairlea, Golden Mile, Oakridge",Bakery,Metro Station,Soccer Field,Ice Cream Shop,Bus Line,Bus Station,Park,Vietnamese Restaurant,Fried Chicken Joint,Fast Food Restaurant
8,M1M,Scarborough,43.716316,-79.239476,1.0,"Cliffcrest, Cliffside, Scarborough Village West",American Restaurant,Motel,Grocery Store,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store
9,M1N,Scarborough,43.692657,-79.264848,1.0,"Birch Cliff, Cliffside West",General Entertainment,Skating Rink,College Stadium,Café,Chinese Restaurant,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store


<h3>Cluster 1</h3>

In [124]:
merged.loc[merged['Cluster Labels'] == 0, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,"Rouge, Malvern",Fast Food Restaurant,Vietnamese Restaurant,Thai Restaurant,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Discount Store,Department Store,Convenience Store


<h3>Cluster 2</h3>

In [125]:
merged.loc[merged['Cluster Labels'] == 1, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,"Guildwood, Morningside, West Hill",Intersection,Rental Car Location,Medical Center,Breakfast Spot,Pizza Place,Mexican Restaurant,Electronics Store,Spa,Department Store,Discount Store
4,Scarborough,Cedarbrae,Hakka Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,College Stadium,General Entertainment
6,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",Discount Store,Department Store,Convenience Store,Bus Station,Coffee Shop,Vietnamese Restaurant,Clothing Store,General Entertainment,Gas Station,Fried Chicken Joint
7,Scarborough,"Clairlea, Golden Mile, Oakridge",Bakery,Metro Station,Soccer Field,Ice Cream Shop,Bus Line,Bus Station,Park,Vietnamese Restaurant,Fried Chicken Joint,Fast Food Restaurant
8,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",American Restaurant,Motel,Grocery Store,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store
9,Scarborough,"Birch Cliff, Cliffside West",General Entertainment,Skating Rink,College Stadium,Café,Chinese Restaurant,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
10,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",Indian Restaurant,Vietnamese Restaurant,Brewery,Pet Store,Chinese Restaurant,Clothing Store,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
11,Scarborough,"Maryvale, Wexford",Middle Eastern Restaurant,Vietnamese Restaurant,Auto Garage,Bakery,Shopping Mall,Sandwich Place,Breakfast Spot,Clothing Store,Fried Chicken Joint,Fast Food Restaurant
12,Scarborough,Agincourt,Clothing Store,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Vietnamese Restaurant,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
13,Scarborough,"Clarks Corners, Sullivan, Tam O'Shanter",Fried Chicken Joint,Thai Restaurant,Fast Food Restaurant,Bank,Intersection,Italian Restaurant,Pizza Place,Noodle House,Gas Station,Chinese Restaurant


In [ ]:
# not list every cluster here